_____________
**Noot**: Het is ten zeerste aangeraden om deze bibliotheek in de [interactieve shell van Python](https://docs.python.org/2/tutorial/interpreter.html#interactive-mode) uit te proberen om de berekeningen mee te volgen.
_____________

# 1. Eindige velden

In [1]:
from FiniteField import *
from util import *

## 1.1 Rekenen modulo $p$

We experimenteren eerst binnen het veld $\mathbb{Z}_5$

In [2]:
Z5 = IntegerField(5)
print("Z5 =", Z5)

Z5 = [0, 1, 2, 3, 4]


Let erop dat dit niet gewoon getallen zijn. Het zijn objecten die er uitzien als een getal wanneer ze afgedrukt worden. Je kunt best variabelen aanmaken voor elk van deze elementen. Merk wel op dat je gewoon kunt itereren over het veld alsof het een lijst zou zijn.

In [3]:
zero, one, two, three, four = tuple([elem for elem in Z5])
# alternatief: zero, one, two, three, four = tuple(Z5.get_elems())
Z5.zero(), Z5.one(), Z5[4]

(0, 1, 4)

Je kunt ook naar elementen verwijzen via hun index. Die index je vind je in de lijst die je ziet als je het veld afdrukt. In een `IntegerField` is `F[i] == i`. In uitbreidingsvelden is dit niet altijd het geval.

In [4]:
print(Z5)
two = Z5[2]
print(two)

[0, 1, 2, 3, 4]
2


Je kunt er wel mee modulorekenen alsof het getallen waren. (Hier: modulo 5)

In [5]:
one + two

3

In [6]:
three * four

2

In [7]:
two / three

4

Ook veelvouden en machten:

In [8]:
2 * three

1

In [9]:
three ** 2

4

Vermenigvuldiging en deling worden efficient uitgevoerd omdat we een generator hebben.

In [10]:
Z5.generator()

2

In [11]:
# even een functie definieren om exponenten te printen
def printpow(base, exponent):
    superscript = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")
    return str(base) + str(exponent).translate(superscript)

g = Z5.generator()
for i in range(len(Z5)):
    print(printpow(g,i), "=", Z5.generator_to_power(i))

2⁰ = 1
2¹ = 2
2² = 4
2³ = 3
2⁴ = 1


Of omgekeerd: de exponent opzoeken met de `generator_exponent`-methode:

In [12]:
for elem in Z5:
    if not elem.is_zero():
        print(elem, "=", printpow(g,Z5.generator_exponent(elem)))

1 = 2⁰
2 = 2¹
3 = 2³
4 = 2²


## 1.2 Veeltermen

We kunnen ook veeltermen definieren. Het $i$-de element dat je meegeeft is de coefficient bij $x^i$ (te tellen vanaf 0).

In [13]:
Polynomial([two, three, zero, one])

2 + 3x + x³

En die hebben hun eigen algebra:

In [14]:
p = Polynomial.one(Z5) + Polynomial.x_to_power(2, Z5)
q = Polynomial([two, two])
print(p)
print(q)
print(p.multiply_x_to_power(5))
print(two * q)
print(p * q)
print(p // q) # deling waarbij de rest genegeerd wordt
print(p % q)

1 + x²
2 + 2x
x⁵ + x⁷
4 + 4x
2 + 2x + 2x² + 2x³
2 + 3x
2


Gehele deling is altijd gedefinieerd, maar let erop dat er een verschil is tussen "/" en "//". Niet alle veeltermen kunnen met "/" door elkaar gedeeld worden.

In [15]:
try:
    p / q
except ValueError as e:
    print(e)

Cannot divide these two polynomials. The remainder is non-zero.


Toepassen op een element is zoals je zou verwachten:

In [16]:
p(one)

2

Nulpunten en factorisatie:

In [17]:
for root in p.find_roots():
    print(root, end=", ")

2, 3, 

In [18]:
factors = p.factor()
print(factors)
print(product(factors) == p)

[2 + x, 3 + x]
True


In [19]:
p.is_reducible()

True

Wat informatie over de coefficienten:

In [20]:
for coef in p:
    print(coef, end=", ")

1, 0, 1, 

Wat op hetzelfde neerkomt als

In [21]:
for i in range(len(p)):
    print(p[i], end=", ")

1, 0, 1, 

In [22]:
len(p), p.degree()

(3, 2)

## 1.3 Uitbreidingsvelden

We zoeken een primitieve veelterm $w(x)$ van graad 2 in $\mathbb{Z}_2[x]$ om het uitbreidingsveld $GF(2^2) = GF(4) \cong \mathbb{Z}_2[x]|_{w(x)}$ te construeren.

In [23]:
Z2 = IntegerField(2)

w, xi_powers = Z2.find_primitive_polynomial(2)
# xi_powers geeft de machten van het primitieve element, uitgedrukt als veeltermen in x
print("w(x) =", w)
for i in range(len(xi_powers)):
    print(printpow("ξ", i), "=", str(xi_powers[i]).replace("x", "ξ"))

w(x) = 1 + x + x²
ξ⁰ = 1
ξ¹ = ξ
ξ² = 1 + ξ


Hiermee kunnen we dus een uitbreidingsveld $GF(2^2) = GF(4)$ construeren.

In [24]:
GF4 = ExtendedField(Z2, 2, "ξ")
GF4

[0, 1, ξ, 1 + ξ]

We kunnen in dit nieuwe veld opnieuw rekenen

In [25]:
xi = GF4.generator()
for i in range(len(GF4)):
    print(xi ** i)

1
ξ
1 + ξ
1


In [26]:
xi + 3 * GF4[3]

1

We kunnen dit veld natuurlijk nog eens uitbreiden tot $GF(4^2) = GF(16)$.

In [27]:
ExtendedField(GF4, 2, greek_alphabet[21])

[0, 1, φ, ξ + φ, ξ + (1 + ξ)φ, 1 + φ, ξ, ξφ, 1 + ξ + ξφ, 1 + ξ + φ, ξ + ξφ, 1 + ξ, (1 + ξ)φ, 1 + (1 + ξ)φ, 1 + ξφ, 1 + ξ + (1 + ξ)φ]

Let erop dat je geen bewerkingen kunt uitvoeren tussen elementen van $GF(2)$ en $GF(4)$.

In [28]:
x = Z2[1]
y = GF4[1]
try:
    print(x + y)
except Exception:
    print("Error: Cannot compute", x, "+", y, ". This is too difficult.")

Error: Cannot compute 1 + 1 . This is too difficult.


Je ziet dat dit niet werkt, ook al zien de elementen er hetzelfde uit. Dit komt doordat de twee elementen achter de schermen anders worden voorgesteld. Om dit op te lossen, kun je de methode `equiv_in_ext_field` gebruiken.

In [29]:
zero_GF4 = Z2.zero().equiv_in_ext_field(GF4)
print(zero_GF4 + GF4[1])

1


En dit is dus wel gedefinieerd.

Bij veeltermen kun je dezelfde methode `equiv_in_ext_field` gebruiken.

In [30]:
zero, one = Z2.get_elems()

p = Polynomial([one, zero, one, one]) # 1x⁰ + 0x¹ + 1x² + 1x³
p_GF4 = p.equiv_in_ext_field(GF4)
print("p(x)     =", p)
print("p_GF4(x) =", p_GF4)
print("p_GF4(ξ) =", p_GF4(xi))

p(x)     = 1 + x² + x³
p_GF4(x) = 1 + x² + x³
p_GF4(ξ) = 1 + ξ


En bij velden ook:

In [31]:
print(Z2)
print(Z2.equiv_in_ext_field(GF4))
# omdat dit een tupel is, kun je heel eenvoudig
# zero, one = Z2.equiv_in_ext_field(GF4) gebruiken

[0, 1]
(0, 1)


In plaats van `equiv_in_ext_field` kun je natuurlijk ook de elementen van $GF(4)$ rechtstreeks aanspreken, zoals hiervoor:

In [32]:
print(GF4)
zero, one, xi = GF4.get_elems()[:3]
print([zero, one, xi])

[0, 1, ξ, 1 + ξ]
[0, 1, ξ]


De inverse functie is `equiv_in_subfield`.

Als je niet meer weet tot welk veld een element behoort, kun je dat opvragen met de variabele `field`. Wijzig deze alsjeblieft niet.

In [33]:
one4 = GF4.one()
one2 = one4.equiv_in_subfield(Z2)
print("one2 ∈", one2.field)
print("one4 ∈", one4.field)

one2 ∈ [0, 1]
one4 ∈ [0, 1, ξ, 1 + ξ]


## 1.4 Factorisatie van $x^n - 1$

We vertrekken met de volgende veelterm in $GF(4)[x]$. Mer op dat aagezien $-1 = 1 \text{ (mod 2)}$, deze veelterm ook uitgedrukt kan worden als $x^n + 1$

In [34]:
n = 5
p = Polynomial.x_to_power(n, GF4) - Polynomial.one(GF4)
p

1 + x⁵

We hebben nood aan een uitbreidingsveld $GF(4^k)$ waarin we een element $\beta$ kunnen vinden dat een primitieve $n$-de wortel uit 1 is. $k$ is hierbij het kleinste getal waarvoor $4^k = 1 \text{ (mod } n \text{)}$

In [35]:
beta, k = GF4.nth_root_of_unity(p.degree(), return_k=True)
# de return_k parameter is optioneel
print("β =",beta, "\nk =", k)

β = ξ + (1 + ξ)α 
k = 2


We zien dat $\beta$ niet in $GF(4)$ gelegen is, maar in $GF(4^k)$. Volgens de theorie zijn alle machten van $\beta$ een oplossing van $p$.

In [36]:
# let op: p is gedefinieerd over GF(4), niet over GF(4^k), waarin β zich bevindt.
P = p.equiv_in_ext_field(beta.field)
all([P(beta ** i).is_zero() for i in range(n+1)])

True

We bepalen de factorisatie met behulp van de cyclotomische nevenklassen modulo $n$ over $GF(4)$.

In [37]:
cyclotomic_cosets(4, n)

[[0], [1, 4], [2, 3]]

Als we deze machten van $\beta$ combineren, hebben we een factorisatie van $x^n - 1$ over $GF(4)$.

In [38]:
factors = GF4.factor_nth_root(n)
for factor in factors:
    print(factor)

1 + x
1 + (1 + ξ)x + x²
1 + ξx + x²


En inderdaad:

In [39]:
product(factors) == p

True

# 2. Codetheorie

## 2.1 Lineaire algebra

In [40]:
from LinearAlgebra import *

Vectoren en matrices zijn vanzelfsprekend zodra je in eindige velden kunt werken.

In [41]:
Z3 = IntegerField(3)
zero, one, two = Z3.get_elems()

In [42]:
v = Vector([one, two, zero, two, zero])
w = Vector([zero, two, zero, zero, one])
v + w

[1, 1, 0, 2, 1]

In [43]:
two * w

[0, 1, 0, 0, 2]

Hamming-gewicht en -afstand:

In [44]:
(v - w).weight(), v.distance(w)

(3, 3)

Indexatie en iteratie:

In [45]:
for i in range(len(v)):
    print(v[i], end=", ")

1, 2, 0, 2, 0, 

In [46]:
for component in v:
    print(component, end=", ")

1, 2, 0, 2, 0, 

In [47]:
v.get_elems()

[1, 2, 0, 2, 0]

De matrices zijn ook van de partij:

In [48]:
A = Matrix([[one, two], [two, zero]])
print(A + A.transpose())

[2, 1]
[1, 0]



In [49]:
for row in A:
    print(Vector(row))

[1, 2]
[2, 0]


Vectoren zijn hier rijmatrices, geen kolommatrices. Dat is van belang voor de vermenigvuldiging.

In [50]:
v = Vector([zero, one])
try:
    print(A * v)
except Exception:
    print("Je moest de volgorde omkeren om te vermenigvuldigen. v.A =", v * A)

Je moest de volgorde omkeren om te vermenigvuldigen. v.A = [2, 0]
